# Симуляция рынка любви или как начинающему сутенёру прокормить свою семью.

### Ярослав Лисняк, BAE'22

#### Постановка задачи:

Наш основной агент -- сутенёр, у которого в распоряжении есть $k$ девушек. Он планирует выйти на Московский рынок. Будем считать, что для этого ему потребуется выбрать местоположение апартаментов каждой из девушек и назначить им цены. При принятии решения он будет руководствоваться только максимизацией общей выручки. 

#### Основные предпосылки модели:

1. Рассматривается только женская проституция. Чрезмерно толерантных читателей попрошу смириться с тем, что мужская часть этого рынка представляет незначительно малую долю, да и не очень интересна для исследования. 

2. Клиенты будут сгенерированы.

У второй предпосылки есть ряд разумных объяснений. В первую очередь возникает серьезная проблема со сбором данных по клиентам. Их персональные данные в промышленных количествах хранятся разве что в базах данных борделей, к которым не подберёшься. Во-вторых, входя на рынок любому, кхм, предпринимателю придётся собственноручно оценивать спрос, чем мы и займёмся как раз при помощи генерации потенциальных клиентов. 

Остальные предпосылки будем вводить уже ближе к делу. Так же по ходу проекта я буду указывать, какие клетки запускать не стоит, а какие нужно. Рекомендую прислушаться. 

## Часть 0.1 From python import brrr brrr. 

In [ ]:
# Basics. 
import random as rand
from itertools import combinations, permutations
import numpy as np   
import pandas as pd  
import time          
import re

# Data mining. 
import requests
import json
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from webdriver_manager.firefox import GeckoDriverManager
from selenium.webdriver.common.by import By
from fake_useragent import UserAgent

# Visualisation, except R.
from tqdm import tqdm
import chart_studio.plotly as py
from plotly.offline import iplot
import cufflinks as cf
import plotly.figure_factory as ff
from matplotlib import pyplot as plt
import plotly.express as px

# Machine learning. 
from sklearn.model_selection import (train_test_split, StratifiedKFold, KFold, cross_val_score)
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import r2_score, make_scorer
from sklearn.preprocessing import LabelEncoder
from sklearn.linear_model import LinearRegression, Lasso, Ridge
from sklearn.model_selection import train_test_split
from catboost import CatBoostRegressor

## Часть 0.2 Использованные технологии:

Чтобы проще было ориентироваться в гигантском проекте, перечислю кратко, что здесь успело произойти:

$\textbf{pandas}$ -- продвинутая обработка данных на протяжении всего процесса, работа с гигантскими датасетами.

$\textbf{веб-скреппинг}$ -- использование селениума и целая борьба с серверами гугла и яндекса, блокирующими автоматизированный сбор данных. (их защиту, таки удалось обойти)

$\textbf{REST API}$ -- исплользовался для того, чтобы вытянуть данные об адресах и станциях метро. 

$\textbf{Визуализация данных}$ -- интерактивные графики на этапе разведочного анализа из библиотек, в том числе не обсуждавшихся в курсе. 

$\textbf{Математические возможности python}$ -- использовались и очень много.

$\textbf{Регулярные выражения}$ -- применялись для обработки адресов и имён маршрутных точек. 

$\textbf{R, tidyverse, ggplot2, extensions}$ -- использовались на довольно продвинутом уровне, все результаты в файле "R_part2_2". 

$\textbf{Дополнительные технологии}$ -- много моделей машинного обучения, в том числе catboost. 

## Часть 1. Сбор данных. 

Для начала подыщем себе подходящий сайт: 

1. Достаточная информативность. (Чтобы хватало разнообразия для моделей)
2. Достоверность анкет. (Не хотелось бы, чтобы вместо принцессы, к нам приезжала жаба)

Чтобы вручную не прописывать путь к веб драйверу, нужно пипнуть соответствующий модуль в терминале: pip install webdriver-manager. 

In [ ]:
from webdriver_manager.chrome import ChromeDriverManager
driver = webdriver.Chrome(ChromeDriverManager().install())

### Неприятные сюрпризы.  Не обязательно запускать.

Изначально, я собирал данные где-то в середине мая, что в итоге обвалило им сайт. Уже ближе к дэдлайну проекта сайт снова начал работать, но, во-первых, они поменяли его архитерктуру, во-вторых, они убрали около 80% анкет. В то же время, я уже успел прогнать большую часть моделей по изначально собранным данным. В этом разделе я немного переписал часть с парсингом, чтобы Вы, мой дорогой читатель, могли убедиться в том, что всё работает. (если они снова не поменяют сайт к моменту проверки, конечно)

Все вычисления же мы будем проделывать на изначально сохранённом датасете girls_old.csv, так как я не хочу заново обучать все модели и причёсывать данные каждый раз, как владельцам сайта что-то взбредёт в голову. 

Итак, парсинг:

In [ ]:
#200. 0к.
page_link = 'https://b.feimsk.com/'
response = requests.get(page_link, headers={'User-Agent': UserAgent().chrome})
response 

In [ ]:
# Построим списки категорий. 
etalon_personal_info = ['Город', 'Район', 'Возраст', "Рост", "Вес", "Размер бюста",
                        "Цвет волос", "За час у нее", "За два часа у нее", "За ночь у нее", "За час у себя",
                        "За два часа у себя", "За ночь у себя"]

etalon_str = "Классика,Анальный,Групповой,Двойное проникновение,С резинкой,Без резинки,Глубокий,В машине,Кунилингус,Анилингус,В рот,На лицо,На грудь,Бандаж,Госпожа,Ролевые игры,Лёгкая доминация,Порка,Рабыня,Фетиш,Трамплинг,Страпон,Игрушки,Профи,Не профи,Классический массаж,Профессиональный,Расслабляющий,Тайский,Урологический,Эротический,Ветка сакуры,Лесби откровенное,Лесби лёгкое,Анальный,Классический,Выдача,Приём,Эскорт,Фото/Видео,Услуги семейной паре "
etalon = list(map(str, etalon_str.split(",")))

# Закодируем для удобства цвет волос.
hair = {"Брюнетка":1,"Шатенка":2,"Блондинка":3,"Русая":4,"Рыжая":5}

In [ ]:
driver.get(page_link)
button = driver.find_element_by_xpath('//*[@id="sectionCatalog"]/div/button[1]')
button.click()

In [ ]:
# Разворачиваем страницу, чтобы охватить как можно больше дам.
for i in tqdm(range(70)):
    driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
    selen_page = driver.page_source
    soup = BeautifulSoup(selen_page,'html.parser')

In [ ]:
# Строим список со всемы ссылками. 
girls_links = soup.findAll('a', attrs = {'target':'_blank'})
girls_links_list = []
for i in girls_links:
    if i['href']!= '/agreement.html':
        girls_links_list.append(i['href'])

Теперь из всей информации, что нам нужно собрать, построим базу датасет, с которым будем работать в дальнейшем. 

In [ ]:
#пустой датасет и туда большим циклом все кидаем. Подробности ниже
df = pd.DataFrame()
for link in tqdm(girls_links_list): #проходим по каждой ссылке
    #варим из ссылки суп
    girls_link = link 
    response = requests.get(girls_link, headers={'User-Agent': UserAgent().chrome})
    html = response.content
    soup = BeautifulSoup(html,'html.parser')
    #суп персональной инфы рост вес город цены и тд
    personal_info = soup.findAll('div', attrs = {'class':'value'}) 
    #делаем лист и кидаем туда всю инфу. Два раза поп - выкинули расу
    personal_info_ls = []
    for i in personal_info:
        personal_info_ls.append(i.text)
    personal_info_ls.pop(1)
    personal_info_ls.pop(7)
    personal_info_ls.pop(7)
    personal_info_ls.pop(7)
    personal_info_ls.pop(7)
    #переводим строки в числа, строку с волосами меняем на категорию
    for i in range(2,len(personal_info_ls)):
        if i==6:
            personal_info_ls[i] = hair[personal_info_ls[i]]
        elif i<6:
            try:
                personal_info_ls[i] = int(personal_info_ls[i])
            except ValueError:
                continue
        elif i>6:
            if personal_info_ls[i] == "-":
                personal_info_ls[i] = np.nan
            else:
                personal_info_ls[i] = int(personal_info_ls[i][:-1])
    #суп из категорий на ссылке
    categorys = soup.findAll('ul', attrs = {'class':'catalog-card__services-list'})
    #лист для записи туда категорий (уже распарсено)
    a = []
    for i in categorys:
        a.append(i.text.split('\n')) #сплитим по разрывам - останется несколько пустых слотов в каждом листе
    for j in range(2): #два раза убираем пустые места, я не знаю почему 2 раза, но работает только так
        for i in range(len(a)):
            a[i].remove('')
    #вектор категорий, в который мы сложим лист с листами переведенный в один сплошной лист,
    #а потом пару категорий переименовали
    categorys_vector = []
    for sublist in a:
        for item in sublist:
            categorys_vector.append(item)
    categorys_vector = [w.replace('Классический', 'Классический массаж') for w in categorys_vector]
    categorys_vector = [w.replace('Лёгкое', 'Лесби лёгкое') for w in categorys_vector]
    categorys_vector = [w.replace('Откровенное', 'Лесби откровенное') for w in categorys_vector]
    #вектор для дамми-кодирования - туда помещаются 0 и 1 - есть ли категория или нет по сравнению с эталоном
    categorys_dummy = []
    for category in etalon:
        if category in categorys_vector:
            categorys_dummy.append(1)
        else:
            categorys_dummy.append(0)
    #сделали словарик и из него датафрейм 
    dictionary = dict(zip(etalon+etalon_personal_info, categorys_dummy+personal_info_ls))
    if df.empty:
        df = pd.DataFrame.from_dict([dictionary])
    else:
        df = df.append(pd.Series(dictionary), ignore_index=True)

In [ ]:
df = df.fillna(df.mean())

In [ ]:
df.to_csv('dummy_info_2.csv') # Сохраняем полученный результат. 

## Часть 2.1 Причёсываем данные и наводим красоту. Начинаем запускать.

Сейчас нам предстоит под разными углами посмотреть на информацию, которую мы вытянули и отследить аномалии в данных, если такие вдруг имеются и в целом оценить на глаз, есть ли какие-то явные взаимосвязи и закономерности. 

In [343]:
gen_df = pd.read_csv('girls_old.csv')

In [344]:
# Построим несколько интерактивных графиков.
cf.go_offline()

# Устанавливаем глобальную тему 
cf.set_config_file(world_readable=True, theme='pearl', offline=True)

In [345]:
gen_df['Вес'].iplot(kind='hist', xTitle='Weight',
                  yTitle='Number', title='Weight distribution')

Первая проблема, с которой мы столкнулись -- не тот тип данных в некоторых колонках, потратим 3 минуты, чтобы исправить это вручную. 

In [346]:
gen_df['За час у нее'] = pd.to_numeric(gen_df['За час у нее'], errors='coerce')
gen_df['Размер бюста'] = pd.to_numeric(gen_df['Размер бюста'], errors='coerce')
gen_df['За два часа у нее'] = pd.to_numeric(gen_df['За два часа у нее'], errors='coerce')
gen_df['За ночь у нее'] = pd.to_numeric(gen_df['За ночь у нее'], errors='coerce')
gen_df['За час у себя'] = pd.to_numeric(gen_df['За час у себя'], errors='coerce')
gen_df['За два часа у себя'] = pd.to_numeric(gen_df['За два часа у себя'], errors='coerce')
gen_df['За ночь у себя'] = pd.to_numeric(gen_df['За ночь у себя'], errors='coerce')

Теперь мы готовы построить один из моих любимых графиков, что поможет нам отловить огромное количество аномалий. 

In [347]:
figure = ff.create_scatterplotmatrix(
    gen_df[['Возраст', 'Вес', 'Рост', 'За час у нее']],
    diag='histogram', size=6, height=800, width=800)
figure

В глаза сразу же бросаются $\textbf{4 всадницы апокалипсиса}$, которые значительно выбиваются из общего распределения показателей. Построенный график интерактивный, поэтому, мой дорогой читатель, можешь рассмотреть эти выбросы подробнее. 

Просто наведя курсор на соответствующую точку убеждаемся в том, что к нам забралась принцесса весом 104кг, при росте, к слову в скромные 160см. Если предположить, что вес указан по ошибке в фунтах, то она хорошо вписывается в общий тренд, однако вряд ли дело в этом. 

$\textbf{Оффтоп:}$ одна из категорий, которую мы будем рассматривать -- это трамплинг. Как говорит Википедия, трамплинг (англ. trampling, букв. вытаптывание) — сексуальная практика, которая заключается в том, что партнер или несколько партнеров (верхние) ходят, стоят, прыгают или танцуют на теле другого (нижнего). Другими словами, это нечто на стыке садо-мазо и футфетиша. В наших данных весомая доля девочек не согласна на эту, казалось бы, безобидную категорию, но только не наша 104-килограммовая принцесса:

In [348]:
gen_df[gen_df['Вес'] == 104].index # Посмотрим на её id в нашей базе данных.

Int64Index([304], dtype='int64')

In [349]:
gen_df.loc[304] # Поглядим на её анкету и убедимся, что она не против топтать своих клиентов, земля им пухом.

Unnamed: 0                     304
Классический                     1
Анальный                         0
Групповой                        0
Двойное проникновение            0
С резинкой                       1
Без резинки                      0
Глубокий                         1
В машине                         0
Кунилингус                       1
Анилингус                        1
В рот                            1
На лицо                          1
На грудь                         1
Бандаж                           1
Госпожа                          1
Ролевые игры                     1
Лёгкая доминация                 1
Порка                            1
Рабыня                           1
Фетиш                            1
Трамплинг                        1
Страпон                          0
Игрушки                          0
Профи                            1
Не профи                         1
Классический массаж              0
Проффессиональный                0
Расслабляющий       

Продолжим наш поход по аномалиям, на очереди у нас стройняшка:

In [350]:
gen_df[gen_df['Вес'] == 3].index

Int64Index([147], dtype='int64')

Учитывая, что на свет люди появляются, как правило с весом, превышающим 3кг, да и младенцев не берут в этот бизнес, уберём стройняшку из данных. Топаем дальше и натыкаемся на барышню в анкету которой судя по всему попали лишние нули:

In [351]:
gen_df[gen_df['За час у нее'] == 30000].index 

Int64Index([0], dtype='int64')

Несмотря на то, что для элитного эскорта цена вышла более, чем правдоподобная, стоит помнить, что мы парсили сайт с услугами более низкого класса, в чём легко убедиться, просто взглянув на то, как сильно цена в 30000 смещает ценовые гистограммы на графике, что мы построили выше. Поэтому, увы, с нашей богатой леди нам тоже предстоит попрощаться. 

В детстве мама меня учила тому, что нужно уважать старших. В случае с четвертой всадницей можно буквально войти в историю. Как-никак 17-й век...

In [352]:
gen_df[gen_df['Возраст'] == 241].index

Int64Index([470], dtype='int64')

In [ ]:
gen_df.drop([0, 147, 304, 470], inplace=True) # Помашем ручкой нашему отряду самоубийц.

Мы убрали выбросы, которые сразу бросаются в глаза. На уже относительно чистых данных построим ещё несколько продвинутых визуализаций, на этот раз уже в R. Для этого нужно будет открыть файл "Part_2.2.rmd" или, так как речь идёт о готовом R markdown "Part_2.2.pdf" (удобства ради). 

## Часть 3. Наивная версия модели.

Как мы уследили в предыдущем разделе, едва ли заметна какая-либо связь между параметрами. Для искушённых отмечу, что да, я понимаю, что для каких-либо ML алгоритмов у нас слишком мало точек. Однако, давайте всё же попробуем попытать удачу и построить пару моделей, которые по набору параметров будут предсказывать цену за час. 

Если предположить, что такой алгоритм вдруг имеет приемлемую предсказательную силу, то нашему сутенёру было бы достаточно просто построить набор предсказаний для каждой из своих барышень, выбрать местоположения, максимизирующие цену за час и воплотить всё в жизнь. 

Как мы увидим ниже, многомерная линейная регрессия споткнулась о распространённую проблему мультиколлинеарности. Мы попробуем воспользоваться встроенной в sklearn регуляризацией, а ещё попробуем исправить ситуацию руками. В конечном итоге мы убедимся, что это не сильно спасает ситуацию, так как коэф. $R^2$ всё ещё будет оставаться очень низким.

Приступим. 

In [ ]:
df = pd.read_csv("Chisto_chisto.csv")
df = df.drop(columns=["Unnamed: 0"])
df = df.drop(columns=["Город"])


le = LabelEncoder()
le.fit(df["Район"])
df["Район_цифры"] = le.transform(df["Район"])

df = df.fillna(df.mean())

X = df.iloc[:,np.append(np.arange(39), 51)]
Y = df.iloc[:,46]

X_train,X_test,Y_train,Y_test = train_test_split(X, Y, test_size = 0.25)

simple = LinearRegression()
ridge = Ridge()
lasso = Lasso()

simple.fit(X_train, Y_train)
ridge.fit(X_train, Y_train)
lasso.fit(X_train, Y_train)

r2_score(Y_test, simple.predict(X_test)), r2_score(Y_test,ridge.predict(X_test)),r2_score(Y_test,lasso.predict(X_test))

Чтобы убрать мультиколлинеарность, склеив признаки, корреляция между которыми превышает, скажем 0.8 и после немного причешем наши данные.

Как я уже говорил, это не сильно поможет ситуации, на что есть ещё ряд причин, которые я не смогу уместить в этот проект, так как он и без этого выходит не особо лаконичным. 

Попробуем более продвинутую модель случайных решающих лесов:

In [ ]:
a = df.corr()
high_corr = []

for i in range(a.shape[0]):
    for j in range(a.shape[0]):
        if a.iloc[i,j]>0.8 and i!=j:
            cross_category = [a.columns[i], a.columns[j]]
            cross_category_invert = [a.columns[j], a.columns[i]]
            if (cross_category_invert not in high_corr) and ('За' not in a.columns[i]):
                high_corr.append(cross_category)
                
for i in range(len(high_corr)):
    new_name = (high_corr[i][0] + ' и ' + high_corr[i][1])
    df[new_name] = df[high_corr[i][0]]+df[high_corr[i][1]]
    df[new_name] = df[new_name].replace({2:1})
    
df1 = df

for i in range(len(high_corr)):
    try:
        df1 = df1.drop(columns=high_corr[i][0])
    except KeyError:
        continue
    try:
        df1 = df1.drop(columns=high_corr[i][1])
    except KeyError:
        continue
        
try:
    df1  = df1.drop(columns="Лесби откровенное")
except KeyError:
    pass

Мне показалось подозрительным то, как стремительно $R^2$ меняет свои значения. Для хорошей модели такого происходить не должно, поэтому я решил провести небольшой эксперимент, зафиксировав количество случайных разбиений для кросс-валидации и замерив $R^2$ для каждого.

Следующий блок кода потребует времени на выполнение. 100 экспериментов требует около 7-8 минут. Если это слишком долго или, вдруг, Ваш комп значительно медленнее, то можете поменять значение переменной num_experiments.

Или же взглянуть на уже посчитанный ранее результат, открыв файл "R2_experiment.png"

#### Time warning: 
Следующая клетка работает довольно долго, на уже готовый результат можно посмотреть в файле: stupid_machines. 

In [ ]:
%%capture
num_experiments = 100
R2_values = []
numbers = np.arange(1, num_experiments + 1)

for i in tqdm(range(num_experiments)):

    X1 = df1.iloc[:,np.append(np.arange(33), np.append(np.arange(34,39), np.arange(45, 5)))]
    Y1 = df1.iloc[:,40]
    X_train1,X_test1,Y_train1,Y_test1 = train_test_split(X1, Y1, test_size = 0.25)

    rf = CatBoostRegressor(iterations=1024)
    rf.fit(X_train1, Y_train1)
    R2_values.append(r2_score(rf.predict(X_test1), Y_test1))

In [ ]:
dataset = pd.DataFrame({'Experiment number': numbers, 'R2 value': R2_values})
fig = px.line(dataset, x='Experiment number', y='R2 value')
fig.show()

Результат просто великолепный с точностью до наоборот! Все эксперименты не зависели друг от друга, поэтому их порядок на данном графике совсем условный. Как мы видим, качество нашей модели ведёт себя, мягко говоря, не адекватно. Волатильности этого случайного блуждания может позавидовать большая часть нестабильных акций. 

С точки зрения предсказательной силы творится полный хаос, поэтому наша наивная модель не сработала. 

В то же время, нужно всё-таки как-то выбрать цены, не делая этого "на глазок". 

## Часть 4. Усложняем модель: задача клиента.

Мы будем генерировать потенциальных клиентов. Для этого нужно понять, чего люди хотят. Вполне естественно сделать это, задав функцию полезности. 

Обычно, когда парень обращается к проститутке, он уже знает, чего примерно он хочет, поэтому предположим, что у каждого клиента есть список того, что он хотел бы реализовать за поездку. Каждому клиенту $i$ сопоставим вектор предпочтений $v_i \in \{0, 1\}^n$, где $n$ -- количество категорий, которые мы включили в анализ. 

При генерации будем считать, что клиенты появляются на свет со случайно сгенерированными по Бернули предпочтениями:

In [ ]:
def preference_generator(n, p): # np -- ожидаемое количество категорий в конкретных предпочтениях. 
    preferences = np.zeros(n)
    
    for i in range(n):
        if rand.random() < p:
            preferences[i] = 1
        else:
            preferences[i] = 0
            
    return preferences

Нам нужно как-то оценивать совместимость произвольного клиента с произвольной ночной бабочкой, поэтому давайте построим какую-нибудь метрику на пространстве бинарных векторов. Обратим внимание на то, что нас не волнует ситуация, когда у барышни доступна категория, не интересная клиенту, хотя обычно такая ситуация должна отдалять векторы друг от друга. 

Чтобы обойти эту проблему, можно рассмотреть скалярное произведение. В таком случае, нас будет интересовать только
количество совпадений по категориям. Но не будем сдаваться, оставив метрику такой банальной. Добавим, например убывающую предельную отдачу от совпадений. Для этого вспомним (или узнаем), что если $\varrho(.)$ -- метрика, а $g$ - скалярная, выпуклая вверх функция, то $g(\varrho(.))$ -- тоже метрика*. 

Большая совместимость будет приносить большую полезность, что логично. Помимо этпоеого нам потребуется функция затрат.

$TC(t, p)$, где t -- стоимость поездки, $p$ -- стоимость часа приключений. (Тут мы предполагаем, что если кому-то требуются услуги больше, чем на час, то ему не обязательно обращатьс в подобные сервисы)

В функции затрат мы разбиваем издержки на 2 переменные, так как если к стоимости поездки мы относимся нейтрально, то стоимость нашей спутницы будет отражать в себе риски. При прочих равных мы бы не хотели обращаться к совсем бюджетным вариантам, так как здоровье всё-таки дороже. Учтём это, сделав предельную отдачу от трат за час убывающей.

(*) Если Вы не верите мне на слово, то приглашаю Вас испытать удовольствие, самостоятельно доказав это утверждение. 

Нам нужно провернуть следующую авантюру: по номеру барышни $j$ нужно рассчитывать $cl_j, p_j, t_j$. С первыми 
двумя переменными разобраться не составит труда, в случае с ними, нужно написать функции вида:

$$p: \mathbb{N} \to \mathbb{R}_+, ~ cl: \mathbb{N} \to \mathbb{R}_+^n$$

С $t_j$ придётся повозиться.  

In [ ]:
df = pd.read_csv("Chisto_chisto.csv")
df = df.drop(columns=["Unnamed: 0"])
df = df.drop(columns=["Город"])

le = LabelEncoder()
le.fit(df["Район"])
df["Район_цифры"] = le.transform(df["Район"])

df = df.fillna(df.mean())

In [ ]:
df["service_value"] = np.arange(len(df))

X = df.iloc[:,:-1]
Y = df["service_value"]
size = 398/len(df) # 398 реальных девушек и 2 наших. 

X_train, Market, Y_train, Y_test = train_test_split(X, Y, test_size = size) # Генерируем случайную подвыборку Market.

In [ ]:
def price(n, arr): 
    return arr['За час у нее'].values[n] # df -- датасет, с которым мы будем работать. 

def cl(n, arr): 
    return arr.iloc[n, 0:39].values

In [ ]:
def convex_function(x): # Какая-нибудь выпуклая функция.
    return x**.5

def metrics_function(v, cl1, f): # Наша метрика.
    return f(v.dot(cl1))         

def greed_function(t, p): # Итоговая функция затрат.
    return t + 60*p**.5

Теперь мы можем поставить задачу клиента:
$$u_i(w_j) = A*g(\varrho(v_i, cl_j) - TC(t_j, p_j) \to \max\limits_{w_j \in W}, ~ A \in \mathbb{R}$$

In [ ]:
def client_happyness(v, cl, t, p, f, TC, TR): # Подбор масштаба осуществляем при помощи гениальности и удачи. 
    PAY = TR(v, cl, f)
    LOSS = TC(t, p)
    return 1000*PAY - LOSS

Нам нужно будет собрать некоторые данные о населении Москвы. Для этого я зашёл на сайт Правительства Москвы и скопировал в excel таблицу с населением каждого района. После этого я посчитал, какую долю в населении составляет каждый район. В дальнейшем мы будем генерировать клиентов случайным образом и вероятностно они будут распределены пропорционально долям населения в районах. Можно считать, что это

$\textbf{Предпосылка:}$ в каждом районе люди хотят любви одинаково. Другими словами, места жительств сгенерированных клиентов распределены пропорционально текущим районным плотностям населения Москвы. 

Я уже подготовил нужный для этого датасет population.csv, скачаем его и почистим от мусора. 

In [ ]:
df = pd.read_csv("population.csv")
df = df.drop(columns=["Unnamed: 2"])
df = df.dropna()

Каждый район нам предстоит аппроксиммировать ближайшей к нему станцией метро, с чем будет связана предстоящая, тяжёлая работа, которая приблизит нас к созданию функции, определяющей стоимости маршрутов. 

Для начала обратимся к гугл-картам. 

In [ ]:
maps = 'https://www.google.com/maps'
driver = webdriver.Firefox(executable_path=GeckoDriverManager().install())    
driver.get(maps)

#### WARNING: НЕ ЗАПУСКАТЬ!

Следующая клетка сильно зависит от мощности компа и загруженности серверов гугл на момент выполнения, поэтому есть риск того, что Вы не сможете выполнить весь цикл. 

Все необходимые данные уже записаны ниже и мы будем пользоваться ими, обращаясь к готовому датасету. 

In [ ]:
subway = []
for i in tqdm(range(len(df))):
    district = df.iloc[i,0]
    district_req = district + ' станция метро'
    district_cell = driver.find_element_by_id('searchboxinput')
    district_cell.clear()
    district_cell.send_keys(district_req)
    search_button = driver.find_element_by_id('searchbox-searchbutton')
    search_button.click()
    time.sleep(2)
    soup = BeautifulSoup(driver.page_source, 'html.parser')
    try:
        subway.append(soup.findAll('h3', attrs = {'class':'section-result-title'})[0].text)
    except:
        subway.append(soup.findAll('h1', attrs = {'class':'section-hero-header-title-title GLOBAL__gm2-headline-5'})[0].text)

После парсинга зачастую приходится чистить данные, так что воспользуемся регулярными выражениями и выгребем всё лишнее. Далее будет много нудной техники, чтобы привести информацию в удобный для работы вид. 

In [ ]:
for i in range(len(subway)):
    subway[i] = re.sub(r"ст. м. ", "", subway[i])
    subway[i] = re.sub(r"Метро", "", subway[i])
    subway[i] = re.sub(r"Станция метро", "", subway[i])
    subway[i] = re.sub(r"станция метро", "", subway[i])
    subway[i] = re.sub(r" «Преображенская пл.»", "Преображенская пл", subway[i])
    subway[i] = re.sub(r" (к/ст, выс.)", "", subway[i])
    subway[i] = re.sub(r' "Крылатское"', "Крылатское", subway[i])
    subway[i] = re.sub(r' "Войковская" (выс.)', "Войковская", subway[i])
    subway[i] = re.sub(r'(Тушинская)', "", subway[i])
    subway[i] = re.sub(r' "Люблино"', 'Люблино', subway[i])
    subway[i] = re.sub(r' Ломоносовский проспект', 'Ломоносовский проспект', subway[i])
    subway[i] = re.sub(r' Новогиреево', 'Новогиреево', subway[i])
    subway[i] = re.sub(r"Станция ", "", subway[i])
    subway[i] = re.sub(r"cтанция ", "", subway[i])
    subway[i] = re.sub(r' Октябрьская', 'Октябрьская', subway[i])
    subway[i] = re.sub(r'Преображенская пл', 'Преображенская площадь', subway[i])

In [ ]:
df['subway'] = np.array(subway) #вытянули в массив и добавили к датасету
df = df.set_index("Муниципальное образование") #задали индексом районы для удобства
df #поглядели

In [ ]:
url = "https://apidata.mos.ru/v1/datasets/1488/rows?api_key=f23630fa434d759635c94f59f0349065"
params = {'format': 'json'}
g = requests.get(url)
subway_ls = []

for i in range(len(g.json())):
    subway_ls.append(g.json()[i]['Cells']['Station'])

not_subway = []

for i in range(len(subway)):
    if subway[i] not in np.array(subway_ls):
        not_subway.append(df.iloc[i,].name)
df = df.drop(not_subway)

In [ ]:
# посмотрим какие станции метро в датасете с районами на самом деле не существуют (гугл карты налажали)
not_subway = []
for i in range(len(subway)):
    if subway[i] not in np.array(subway_list["Station"]):
        not_subway.append(df.iloc[i,].name)
        
# можно видеть отдельные районы, которые лежат далеко от Москвы, их можно спокойно выкинуть
df = df.drop(not_subway)

### Продолжаем запускать отсюда:

In [ ]:
df = pd.read_csv("subways_complete.csv")
ladies = pd.read_csv("Chisto_chisto.csv", index_col= 0)

In [ ]:
#смотри, сынок, тут происходит волшебство вращения пандаса
path = pd.DataFrame(np.zeros((df["subway"].unique().shape[0],ladies["Район"].unique().shape[0])))
path.columns = ladies["Район"].unique()
path["start"] = df["subway"].unique()
path = path.set_index("start")

Страшное впереди. Теперь решим следующую задачу. У нас есть два массива: места жительства клиентов и апартаменты наших принцесс. Мы хотим построить матрицу, в ячейке $(i, j)$, которой будет отражена цена поездки на яндекс такси класса "комфорт" из места жительства с номером $i$ к апартаментам с номером $j$. Для этого мы написали необходимые функции, завели массивы и в дальнейшем доберемся до клетки, которая реализует описанный план. 

In [ ]:
driver = webdriver.Firefox(executable_path=GeckoDriverManager().install())    
ref = 'https://taxi.yandex.ru'
driver.get(ref)

In [ ]:
def Comfort(starting_point, end_point): # возвращает стоимость поездки между точками i и j
    
    starto = driver.find_element_by_id('address-a')
    finito = driver.find_element_by_id('address-b')
    starto.clear()
    finito.clear()
    starto.send_keys(starting_point)

    
    
    done_a = False
    time_sleep_a = 0.5
    while done_a != True:
        time_sleep_a +=0.5
        try:
            choice_list_a = driver.find_element_by_id('address-a-popup')
            time.sleep(time_sleep_a)
            choice_list_a.click()
            done_a=True
        except:
            pass
        if time_sleep_a == 5:
            return np.nan
        
    finito.clear()
    finito.send_keys(end_point)
    
    
    done_b = False
    time_sleep_b = 0
    while done_b != True:
        time_sleep_b +=0.5
        try:
            choice_list_b = driver.find_element_by_id('address-b-popup')
            time.sleep(time_sleep_b)
            choice_list_b.click()
            done_b=True
        except:
            pass
        if time_sleep_b == 5:
            return np.nan
        
    done_parse = False
    time_sleep_parse = 0
    while done_parse != True:
        time_sleep_parse +=0.5
        try:
            time.sleep(time_sleep_parse)
            soup = BeautifulSoup(driver.page_source, 'html.parser')
            comfort = soup.findAll('div', attrs = {'class':"TariffCard__price isFixed"})[1].text
            done_parse = True
        except:
            pass
        if time_sleep_parse == 5:
            return np.nan
    
    comfort = float(comfort.replace("\u2006₽", ""))
    return (comfort)

#### WARNING: НЕ ЗАПУСКАТЬ!

Следующая клетка активирует наш алгоритм и начинает создавать матрицу стоимостей маршрутов. Что стоит об этом знать: 

1) Алгоритм выполняется приблизительно 9 часов. 

2) Алгоритм может сжечь ваш комп. 

Теперь немного подробнее. Длительное время выполнения объясняется тем, что сервера яндекса защищены от того, чтобы их так активно и автоматизировано парсили, из-за чего необходимо искусственно выставлять задержки на некоторые операции. Как Вы можете видеть, я прописал ветвистую структуру из экспепшенов, которые регулируют time.sleep(). 

Для компа вредно такое количество операций, так как он может начать перегреваться. Дело в том, что если во многих алгоритмах я могу поспользоваться производственными мощностями гугла через colab, то когда речь идёт о парсинге через селениум, у меня этого сделать не получается, потому что гугловское железо не хочет открывать у себя управляемый браузер. В связи с этим мне пришлось оставлять свой комп. 

$\textbf{Ещё один подводный камень}$ в том, что даже если со всеми предпринятыми мерами цикл рухнет, промежуточный результат сохранится и можно будет запустить всё заново, начав с того индекса, на котором всё сломалось, без потери результата. Этот процесс тоже можно было бы автоматизировать, но у меня всё сработало и так. 

In [ ]:
for i in tqdm(range(path.shape[0])):
    start = 'метро ' + path.iloc[i,:].name
    for j in range(path.shape[1]):
        destination = 'метро ' + path.columns[j]
        price = Comfort(start, destination)
        path.iloc[i,j] = price
    print (i) #следим за ходом цикла
    path.to_csv("path.csv") #и сохраняемся если вдруг упадет интернет или еще чего

### Продолжаем запускать отсюда:

Теперь предстоит сгенерировать базу клиентов. 
Не люблю искать математические библиотеки для задач, для которых можно за пару минут написать необходимые функции своими руками, так что сейчас мы реализуем функцию произвольного конечного дискретного распределения: 

In [ ]:
# датасет с распределением людей по районам. 
new_sub = pd.read_csv("obriz_-_Sheet1.csv")
subway_p = pd.DataFrame(new_sub.groupby("subway")["Доля"].sum())
subway_p["subways"] = subway_p.index
subway_p["index"] = np.arange(len(subway_p))
subway_p = subway_p.set_index("index")

In [ ]:
def partial_sums(arr):
    partial_sums = []
    slave = 0
    for i in range(len(arr)): 
        slave -=- arr[i]
        partial_sums.append(slave)
    return partial_sums

def metro_selector(arr):
    x = rand.random()
    for i in range(len(arr)):
        if x <= partial_sums(arr)[i]:
            return i

In [ ]:
# Функция, генерирующая одного клиента
def client_generator(n,p):
    preferences = preference_generator(n, p)
    geoposition_index = metro_selector(subway_p["Доля"])
    result = np.concatenate((preference_generator(n, p), 
                             np.array(int(metro_selector(subway_p["Доля"]))).reshape(1,)))
    result = result.astype(int)
    return result
# Обращаю внимание, что в результирующем векторе станция метро - это индекс к датафрейму

In [ ]:
# Теперь сделаем функцию, которая принимает количество клиентов и заселяет датафрейм нашей сгенерированной 
# клиентской базой. 

def df_client(n:"количество нагенерированных клиентов", num:"Кол-во категорий", p:"вероятность единички"):
    df = pd.DataFrame()
    for i in range(n):
        if df.empty:
            df = pd.DataFrame(client_generator(num,p).reshape(1,-1))
        else:
            dop = (client_generator(num,p).reshape(1,-1))
            df = df.append(pd.DataFrame(dop))
    return df

Наша клиентская база по сути готова, теперь нужно разобраться с тем, чтобы выбрать подмножество нашего исходного датасета с барышнями и добавлять в него девушек, которые будут нам приносить прибыль. На этом моменте я сразу оговорюсь о том, что количество операций алгоритма, к которому мы идём можно оценить снизу следующим образом:

$$ A_n^k * K * W $$

Где $n = 90$ количество станций, где проживают барышни, $k = 2$ -- наши девочки, $K = 5000$ количество сгенерированных клиентов, $W = 400$ -- количество девушек на всём рынке, что мы моделируем. Подставив эти значения получим нижнюю оценку в $90\times 89\times 400 \times 5000 = 16020000000$. 16 миллиардов операций -- многовато, надеюсь, что до дэдлайна мой комп всё таки умудрится рассчитать всё необходимое. 

Как видите, мы довольно сильно ограничены вычислительными мощностями, поэтому будем подстраиваться под ситуацию, возможно нам придется отказаться от наших стартовых значений, которые несомненно, хорошо бы укладывались, с теоретической точки зрения. 

In [ ]:
def converter(nparray, sample_p): # Вспомогательная функция, принимающая на вход список категорий нашей барышни.
    
    x = np.array([2, 2, 2, 2, 2, 2]) # Заполнили ненужный пропуски.
    u = np.array([2, 2, 2, 2, 2]) # Заполнили ненужный пропуски.
    z = np.concatenate((nparray, x), axis=0)
    d = np.concatenate((z, [sample_p]), axis=0) # приклеили цену на 45 место
    m = np.concatenate((d, u), axis=0)
    
    return np.concatenate((m, [777]), axis=0) # приклеили район

Теперь напишем генератор размещений по станциям, считая, что станции закодированы натуральными числами.

In [ ]:
def girls_placement_generator(n, k): # Число размещений k девушек по n станциям.
    List = []
    for elem in list(combinations(range(n), k)):
        for perm in list(permutations(elem)):
            List.append(perm)
    return List

Теперь смоделируем подвыборку из данных, которые мы собирали и причесывали в первых двух частях. 

In [ ]:
path_matrix = pd.read_csv('path_matrix.csv')
path_matrix = path_matrix.fillna(path_matrix.mean())
path_matrix = path_matrix.iloc[np.arange(82), :]
path_matrix = path_matrix.sort_values(by='start')
path_matrix.index = np.arange(len(path_matrix))

Сгенерируем наших клиентов, записав их в client_list: 

$\textbf{Спойлер!}$ мы всё таки проиграли времени своими мощностями, поэтому будем проводить "маленький эксперимент" из 10 клиентов, 2 барышень с изначального рынка и двух наших девушек. 

In [ ]:
client_list = df_client(10, 39, .4)

Теперь мы хотим определить клиента по его номеру: client_id и далее сопоставить ему наиболее подходящую даму на ночь.

In [ ]:
def client_choice(girlswhorelist, client_list, client_id): # Функция, которая принимает id клиента и датасет. 
    preferences = client_list.iloc[client_id].values[:-1]
    geoposition = int(client_list.iloc[client_id].values[-1])
    indicator = -9999999999
    choice = -1
    
    for girl_number in range(len(girlslist)):
        
        hui = girlslist.iloc[girl_number, :]['Район'] # string
        t = path_matrix.iloc[geoposition, ][hui] # Заменить на попозже. 
        p = price(girl_number, girlslist)
        cl1 = cl(girl_number, girlslist)
        
        sexy_value = client_happyness(preferences, cl1, t, p, convex_function, greed_function, metrics_function)
        
        if  sexy_value > indicator:
            indicator = sexy_value
            choice = girl_number
    return choice # Возвращает best response нашего клиента. 

In [ ]:
def garem_generator(k, sample_p): # Выращиваем личный легион длинноногих. 
    garem = []
    for i in range(k):
        garem.append(converter(preference_generator(39, .8), sample_p))
    return garem

Оптимальной стратегией будет назначать матожидание цены за час, считая выборкой изначальный рынок. 
Содержательно это объясняется тем, что во второй и третьей частях мы не нашли никакого паттерна в данных.

In [ ]:
sample_p = Market['За час у нее'].values.mean()                                                    

In [ ]:
# check_Market = Market.append(pd.DataFrame(garem_generator(2, sample_p), columns=Market.columns), ignore_index=True) 
# Приклеили сгенерированных дам к нашей подвыборке. 

Готовим наши размещения. 

In [ ]:
k = 2
Placement_combinations = girls_placement_generator(90, k)

Функция, которая принимает на вход номер размещения и применяет его к рыночному датасету после добавления наших принцесс.

In [ ]:
def girls_placement(number_of_placement, check_Market):  # По номеру размещения 
    for x in range(2, 4): # тут нужно прицелиться на размер нашей выборки. 
        position = Placement_combinations[number_of_placement][3 - x]
        check_Market.iloc[x, -1] = position
        hood = le.inverse_transform([position])[0]
        check_Market.iloc[x, 39] = hood

In [ ]:
def revenue(girlslist, sample_p, client_list): # Функция, пробегающаяся по всем клиентам и считающая нашу выручку. 
    revenue = 0
   
    subindex = 0
    
    for client_number in range(len(client_list)): 
        choice = client_choice(girlslist, client_list, subindex) 
        subindex -=- 1
        
        if choice > 2:
            revenue -=- sample_p

        
    return revenue

In [ ]:
SUPEROBRIZ = check_Market[396:] # Вынужденная мера за час до дэдла... 2 наших девушки, против двух рыночных. 

Финальная часть: программа, которая запускает весь процесс симуляции. Мы будем перебирать все возможные размещения наших барышень по апартаментам и смотреть, какой выбор сделают сгенерированные клиенты, после мы будем сравнивать полученную выручку с предыдущими вариантами и запоминать оптимальный. Массив progress будет отражать рост нашей выручки по мере выполнения алгоритма.

In [ ]:
k = 2
m  = 39
p = 0.8
n = 90

garem = garem_generator(2, sample_p) # Наши девочки. 
check_Market = Market # Изначальное состояние рынка. 
    
TR = 0 # Отправная точка в наших доходах. 
progress = [] # Будем отслеживать, как нас проталкивает вперёд наш алгоритм. 
index = 0 # Вспомогательная переменная, которая существует лишь потому что у меня кончается время на этот проект. 
Profits = [] # Посмотрим на случайные блуждания наших прибылей по ходу перебора. 
p_mean = sample_p 


for placement in tqdm(girls_placement_generator(n, k)): # n -- количество станций метро, k -- количество девочек.

    check_Market = Market.append(pd.DataFrame(garem, columns=Market.columns), ignore_index=True)
    girls_placement(index, SUPEROBRIZ)
    index -=- 1
    
    PROFIT = revenue(SUPEROBRIZ, p_mean, client_list)
    Profits.append(PROFIT)
    
    if PROFIT > TR:
        TR = revenue(SUPEROBRIZ, p_mean, client_list)
        progress.append(TR)

In [341]:
progress

[6765.136578007578, 13530.273156015155, 16912.841445018945, 20295.409734022734]

### Лирическое отступление: 
Изначально я задумывал прогонять проект на наиболее реалистичной выборке: взять около 500 барышень, сгенерировать личный легион длинноногих из примерно 7-10 девушек и сгенерировать значительное число клиентов, порядка 10 или 100 тысяч, после, наблюдать монотонный рост выручки по мере исполнения алгоритма. У меня так же была гипотеза о том, что доля расстановок, которые нужно проверить будет являться величиной первого порядка малости по мере роста числа "наших" девушек и соразмерного роста прочих переменных. Так как при достаточно больших значениях мы будем иметь большие скачки выручки вначале и каждое следующее продвижение будет даваться с всё большими времязатратами. Другими словами, хвостом перебора, доля которого стремится к единице можно будет пренебречь, получив выручку, близкую к оптимальной. 

Для проверки подобного результата потребуется гораздо больше времени на выполнение алгоритма, поэтому закончим на том, что он работает правильно и выдаёт все необходимые для изначальной цели значения. 

Спасибо за внимание, надеюсь, тебе, мой дорогой читатель, было интересно.